##Part 3
#Automatic detection of fraudulous signals using Machine Learning and Deep Learning 

## MVA Project for the course "Deep Learning and Signal detection: Introduction and industrial applications" 

# We create a RNN baseline with fixed batch sequence length

### We first look at the data and see the smallest size of a signal in train and test datasets to set the maximum size of signals we would like to look at.

In [ ]:
import torch
import numpy as np
from glob import glob

In [ ]:
# Creating the list of files of our datasets
files_train = glob("train/*.npz")
files_test = glob("test/*.npz")
files = files_train + files_test

In [ ]:
# We initialize the min_length to the length of the first file in our datasets
min_length = np.load(files[0], mmap_mode='r')['date'].shape[0]

# Loop through all files from test and train set
for file in files:
    length = np.load(file, mmap_mode='r')['date'].shape[0] # Read mmap_mode faster
    if length < min_length:
        min_length = length

min_length

458

### Getting stats from training set for normalization

In [ ]:
# We select only largeur, frequence and puissance features
date = np.array([])
largeur = np.array([])
frequence = np.array([])
puissance = np.array([])

for file in files_train:
    date = np.concatenate((date, [0], np.diff(np.load(file, mmap_mode='r')['date'])))
    largeur = np.concatenate((largeur, np.load(file, mmap_mode='r')['largeur']))
    frequence = np.concatenate((frequence, np.load(file, mmap_mode='r')['frequence']))
    puissance = np.concatenate((puissance, np.load(file, mmap_mode='r')['puissance']))

mean_date = np.mean(date)
mean_largeur = np.mean(largeur)
mean_frequence = np.mean(frequence)
mean_puissance = np.mean(puissance)

std_date = np.std(date)
std_largeur = np.std(largeur)
std_frequence = np.std(frequence)
std_puissance = np.std(puissance)

In [ ]:
mean_date, mean_largeur, mean_frequence, mean_puissance, std_date, std_largeur, std_frequence, std_puissance

(5.875007060286158,
 0.0871414782687525,
 1.9893857392132595,
 -154.90684374070747,
 8.705384067948797,
 0.0845755502639984,
 0.8923242781137891,
 19.407849095325062)

In [ ]:
mean_tensor = torch.tensor([5.875007060286158, 0.0871414782687525, 1.9893857392132595, -154.90684374070747], dtype=torch.float32)
std_tensor = torch.tensor([8.705384067948797, 0.0845755502639984, 0.8923242781137891, 19.407849095325062], dtype=torch.float32)

### The minimum length is 458, so we will take a random sample from each signal of size 458

In [ ]:
from torch.utils.data import Dataset
from random import randint
import json

# We create our file loader for both 
class NPZLoader(Dataset):
    def __init__(self, mode, min_length=458):
        self.mode = mode # train, val or test
        self.dataset = "train" if self.mode in ["train", "val"] else "test"
        self.files = glob(f'{self.dataset}/*.npz')
        
        if self.mode == "train":
            self.files = self.files[:int(0.80 * len(self.files))]
        elif self.mode == "val":
            self.files = self.files[int(0.80 * len(self.files)):]
        
        self.min_length = min_length
        
        with open(f"{self.dataset}_labels.json") as f:
            self.labels = json.load(f)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, item):
        
        npz = np.load(str(self.files[item]), mmap_mode='r')
        length = npz["date"].shape[0]
        random_min_index = randint(0, length - self.min_length) # We take a random index in the range of possible ones

        if self.mode == 'train':
            np_array = np.stack((np.concatenate(([0], np.diff(npz["date"][random_min_index: random_min_index + self.min_length]))),
                                 npz["largeur"][random_min_index: random_min_index + self.min_length], 
                                 npz["frequence"][random_min_index: random_min_index + self.min_length], 
                                 npz["puissance"][random_min_index: random_min_index + self.min_length]), 
                                axis=1).astype(np.float32)
        
        else:
            np_array = np.stack((np.concatenate(([0], np.diff(npz["date"]))),
                                 npz["largeur"], 
                                 npz["frequence"], 
                                 npz["puissance"]), 
                                axis=1).astype(np.float32)
        
        # Normalizing
        torch_array = (torch.from_numpy(np_array) - mean_tensor) / std_tensor
            
        index = self.files[item].split('.')[0].split("-")[1]
        label = torch.FloatTensor([0]) if self.labels[f"pdw-{index}"] == 'nonmenace' else torch.FloatTensor([1])
        
        return torch_array, label

# We define the RNN baseline model

In [ ]:
import torch.nn as nn

In [ ]:
class RNNBaseline(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNNBaseline, self).__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.rnn = nn.RNN(self.input_size, 
                          self.hidden_dim, 
                          self.n_layers, 
                          batch_first=True, 
                          bidirectional=False, 
                          nonlinearity='relu')
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigm = nn.Sigmoid()

    def forward(self, x):
        out, hidden = self.rnn(x)
#         out = out[:, -1, :].contiguous().view(-1, self.hidden_dim) # Take only last timestep
        out = out.max(axis=1)[0] # Maxpooling
        out = self.fc(out)
        out = self.sigm(out)
        
        return out

### Utils

In [ ]:
from tqdm.notebook import tqdm
import torch.optim as optim
import os
from sklearn.metrics import classification_report, confusion_matrix

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1)

# Create experiment folder
if not os.path.isdir("experiment"):
    os.makedirs("experiment")

criterion = torch.nn.BCELoss()

def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(tqdm(train_loader, leave=False)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    scheduler.step()
        
def validation(model, loader):
    model.eval()
    validation_loss = 0
    correct = 0
    y_pred = []
    y_correct = []
    
    for data, target in tqdm(loader, leave=False):
        data, target = data.to(device), target.to(device)
        output = model(data)
        validation_loss += criterion(output, target).data.item()
        # get the index of the max log-probability
        pred = (output >= 0.5)
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        y_correct.append(target)
        y_pred.append(pred)
    
    y_correct = torch.stack(y_correct).cpu().detach().numpy().squeeze()
    y_pred = torch.stack(y_pred).cpu().detach().int().numpy().squeeze()
    print(confusion_matrix(y_correct, y_pred))
    print(classification_report(y_correct, y_pred))
    
    validation_loss /= len(loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        validation_loss, correct, len(loader.dataset),
        100. * correct / len(loader.dataset)))

### Hyperparameters

In [ ]:
input_size = 4
output_size = 1
hidden_dim = 64
n_layers = 8

n_epochs = 32

In [ ]:
# Neural network and optimizer
from torch.optim.lr_scheduler import StepLR

model = RNNBaseline(input_size, output_size, hidden_dim, n_layers)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=8, gamma=0.1, verbose=True) 

# Data initialization and loading
train_loader = torch.utils.data.DataLoader(
    NPZLoader("train"),
    batch_size=32, 
    shuffle=True, 
    num_workers=0)

val_loader = torch.utils.data.DataLoader(
    NPZLoader("val"),
    batch_size=1, 
    shuffle=True, 
    num_workers=0)

# Training
for epoch in tqdm(range(n_epochs)):
    #print(f"Starting epoch {epoch}")
    train(model, epoch)
    validation(model, val_loader)
#     model_file = "experiment" + '/model_' + str(epoch) + '.pth'
#     torch.save(model.state_dict(), model_file)
#     print(f"Model {model_file} saved !")

Adjusting learning rate of group 0 to 1.0000e-03.


  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 1.0000e-03.


  0%|          | 0/400 [00:00<?, ?it/s]

...

[[161  46]
 [ 28 165]]
              precision    recall  f1-score   support

         0.0       0.85      0.78      0.81       207
         1.0       0.78      0.85      0.82       193

    accuracy                           0.81       400
   macro avg       0.82      0.82      0.81       400
weighted avg       0.82      0.81      0.81       400


Validation set: Average loss: 0.3810, Accuracy: 326/400 (81.50%)


  0%|          | 0/50 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 1.0000e-07.


  0%|          | 0/400 [00:00<?, ?it/s]

[[161  46]
 [ 28 165]]
              precision    recall  f1-score   support

         0.0       0.85      0.78      0.81       207
         1.0       0.78      0.85      0.82       193

    accuracy                           0.81       400
   macro avg       0.82      0.82      0.81       400
weighted avg       0.82      0.81      0.81       400


Validation set: Average loss: 0.3810, Accuracy: 326/400 (81.50%)


In [ ]:
# Testing
test_loader = torch.utils.data.DataLoader(
    NPZLoader("test"),
    batch_size=1, 
    shuffle=True, 
    num_workers=0)

validation(model, test_loader)

  0%|          | 0/800 [00:00<?, ?it/s]

[[300 115]
 [ 79 306]]
              precision    recall  f1-score   support

         0.0       0.79      0.72      0.76       415
         1.0       0.73      0.79      0.76       385

    accuracy                           0.76       800
   macro avg       0.76      0.76      0.76       800
weighted avg       0.76      0.76      0.76       800


Validation set: Average loss: 0.4385, Accuracy: 606/800 (75.75%)


# We create a new model robust to batch sequence length change

In [ ]:
from torch.utils.data import Dataset
from random import randint
import json
import torch
import torch.nn as nn
from glob import glob
import numpy as np

# We create our file loader for both 
class NPZAdaptedDataset(Dataset):
    def __init__(self, mode):
        self.mode = mode # train, val or test
        self.dataset = "train" if self.mode in ["train", "val"] else "test"
        self.files = glob(f'{self.dataset}/*.npz')
        
        self.lengths = []
        
        if self.mode == "train":
            self.files = self.files[:int(0.80 * len(self.files))]
            for file in self.files:
                self.lengths.append(np.load(file, mmap_mode='r')['date'].shape[0])
                
        elif self.mode == "val":
            self.files = self.files[int(0.80 * len(self.files)):]
            for file in self.files:
                self.lengths.append(np.load(file, mmap_mode='r')['date'].shape[0])
                
        else: # mode test
            self.files = glob(f'{self.dataset}/*.npz')
            for file in self.files:
                self.lengths.append(np.load(file, mmap_mode='r')['date'].shape[0])
                
        with open(f"{self.dataset}_labels.json") as f:
            self.labels = json.load(f)
            
    def __len__(self):
        return len(self.files)

    def __getitem__(self, item):
        npz = np.load(str(self.files[item]), mmap_mode='r')
        length = npz["date"].shape[0]
        
        np_array = np.stack((np.concatenate(([0], np.diff(npz["date"]))),
                             npz["largeur"], 
                             npz["frequence"], 
                             npz["puissance"]), 
                            axis=1).astype(np.float32)
                
        torch_array = (torch.from_numpy(np_array) - mean_tensor) / std_tensor
        
        index = self.files[item].split('.')[0].split("-")[1]
        label = torch.FloatTensor([0]) if self.labels[f"pdw-{index}"] == 'nonmenace' else torch.FloatTensor([1])
        
        return torch_array, label, length

In [ ]:
class RNNAdapted(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNNAdapted, self).__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.rnn = nn.RNN(self.input_size, 
                          self.hidden_dim, 
                          self.n_layers, 
                          batch_first=True, 
                          bidirectional=False, 
                          nonlinearity='relu')
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigm = nn.Sigmoid()

    def forward(self, x):
        out, hidden = self.rnn(x)
#         out = out[:, -1, :].contiguous().view(-1, self.hidden_dim) # Take only last timestep
        out = out.max(axis=1)[0] # Maxpooling
        out = self.fc(out)
        out = self.sigm(out)
        
        return out

In [ ]:
from tqdm.notebook import tqdm
import torch.optim as optim
import os


device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1)

# Create experiment folder
if not os.path.isdir("experiment"):
    os.makedirs("experiment")

criterion = torch.nn.BCELoss()

def train(model, epoch, loader, scheduler):
    model.train()
    train_loss = 0
    correct = 0
    
    for batch_idx, (data, target) in enumerate(tqdm(train_loader, leave=False)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss.data.item()
        loss.backward()
        optimizer.step()
        
        pred = (output > 0.5)
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    train_loss /= len(loader.dataset)
    
    print('\nTraining set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        train_loss, correct, len(loader.dataset),
        100. * correct / len(loader.dataset)))
    scheduler.step()
        
def validation(model, loader):
    model.eval()
    validation_loss = 0
    correct = 0
    y_pred = []
    y_correct = []
    
    
    for data, target, _ in tqdm(loader, leave=False):
        data, target = data.to(device), target.to(device)
        output = model(data)
        validation_loss += criterion(output, target).data.item()
        # get the index of the max log-probability
        pred = (output > 0.5)
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        y_correct.append(target)
        y_pred.append(pred)
    
    y_correct = torch.stack(y_correct).cpu().detach().numpy().squeeze()
    y_pred = torch.stack(y_pred).cpu().detach().int().numpy().squeeze()
    print(confusion_matrix(y_correct, y_pred))
    print(classification_report(y_correct, y_pred))
    
    validation_loss /= len(loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        validation_loss, correct, len(loader.dataset),
        100. * correct / len(loader.dataset)))

In [ ]:
input_size = 4
output_size = 1
hidden_dim = 64
n_layers = 8

n_epochs = 32

In [ ]:
from random import randint

def collate_fn(data):
    with torch.no_grad():
        signals, labels, lengths = zip(*data)
        min_length = min(lengths)

        random_samples = []

        for signal in signals:
            random_min_index = randint(0, signal.shape[0] - min_length)
            random_samples.append(signal[random_min_index: random_min_index + min_length])

        new_samples_from_signals = torch.stack(random_samples, dim=0)
        new_labels = torch.stack(labels, dim=0)
    
    return new_samples_from_signals, new_labels

In [ ]:
# Neural network and optimizer
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt

model = RNNAdapted(input_size, output_size, hidden_dim, n_layers)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3, capturable=False)
scheduler = StepLR(optimizer, step_size=8, gamma=0.1, verbose=True) 

# Data initialization and loading
train_loader = torch.utils.data.DataLoader(
    NPZAdaptedDataset("train"),
    batch_size=32, 
    shuffle=True, 
    collate_fn=collate_fn)

val_loader = torch.utils.data.DataLoader(
    NPZAdaptedDataset("val"),
    batch_size=1, 
    shuffle=True)

# Training
for epoch in tqdm(range(n_epochs)):
    #print(f"Starting epoch {epoch}")
    train(model, epoch, train_loader, scheduler)
    validation(model, val_loader)
#     model_file = "experiment" + '/model_' + str(epoch) + '.pth'
#     torch.save(model.state_dict(), model_file)
#     print(f"Model {model_file} saved !")

Adjusting learning rate of group 0 to 1.0000e-03.


  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


Training set: Average loss: 0.0203, Accuracy: 998/1600 (62.38%)
Adjusting learning rate of group 0 to 1.0000e-03.


  0%|          | 0/400 [00:00<?, ?it/s]

...

[[149  58]
 [ 18 175]]
              precision    recall  f1-score   support

         0.0       0.89      0.72      0.80       207
         1.0       0.75      0.91      0.82       193

    accuracy                           0.81       400
   macro avg       0.82      0.81      0.81       400
weighted avg       0.82      0.81      0.81       400


Validation set: Average loss: 0.4026, Accuracy: 324/400 (81.00%)


  0%|          | 0/50 [00:00<?, ?it/s]


Training set: Average loss: 0.0120, Accuracy: 1285/1600 (80.31%)
Adjusting learning rate of group 0 to 1.0000e-07.


  0%|          | 0/400 [00:00<?, ?it/s]

[[149  58]
 [ 18 175]]
              precision    recall  f1-score   support

         0.0       0.89      0.72      0.80       207
         1.0       0.75      0.91      0.82       193

    accuracy                           0.81       400
   macro avg       0.82      0.81      0.81       400
weighted avg       0.82      0.81      0.81       400


Validation set: Average loss: 0.4023, Accuracy: 324/400 (81.00%)


In [ ]:
# Testing
test_loader = torch.utils.data.DataLoader(
    NPZAdaptedDataset("test"),
    batch_size=1, 
    shuffle=True)

validation(model, test_loader)

  0%|          | 0/800 [00:00<?, ?it/s]

[[281 134]
 [ 58 327]]
              precision    recall  f1-score   support

         0.0       0.83      0.68      0.75       415
         1.0       0.71      0.85      0.77       385

    accuracy                           0.76       800
   macro avg       0.77      0.76      0.76       800
weighted avg       0.77      0.76      0.76       800


Validation set: Average loss: 0.4554, Accuracy: 608/800 (76.00%)


In [ ]:
sum([element.numel() for element in model.parameters()])

62785